![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [ ]:
# Start your code here!
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats.distributions import norm
from statsmodels.graphics.gofplots import qqplot

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

women = pd.read_csv('women_results.csv', index_col=0, parse_dates=['date'])
men = pd.read_csv('men_results.csv', index_col=0, parse_dates=['date'])

In [ ]:
# Peform exploratory data analysis
def eda_df(dataframe):
    print('SHAPE'.center(50, '='))
    print(dataframe.shape)
    print('INFO'.center(50, '='))
    print(dataframe.info())
    print('TOTAL NULL VALUES (IF ANY)'.center(50, '='))
    print(dataframe.isna().sum())
    print('PREVIEW'.center(50, '='))
    print(dataframe.head())

    for col in dataframe.columns:
        print(dataframe[col].value_counts(normalize=True))

eda_df(women)
eda_df(men)

In [ ]:
# Filtering only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01` as per project scope

def filter_compute_df(df):
    filtered_df = df.loc[(df['tournament'] == 'FIFA World Cup') & (df['date'] >= '2002-01-01')]
    filtered_df['total_goals_scored'] = filtered_df['home_score'] + filtered_df['away_score']
    return filtered_df

filtered_women = filter_compute_df(women)
filtered_men = filter_compute_df(men)

In [ ]:
eda_df(filtered_women)
eda_df(filtered_men)

In [ ]:
# Test for Normality using Shapiro-Wilk Test and QQPlot
def shapiro_qqplot(data, alpha):
    stat, p = shapiro(data)
    print(f"p-value: {p} test stat: {round(stat, 4)}")
    if p <= alpha:
        print(f"p-value < {alpha}, distribution is not normal")
    
    else:
        print(f"p-value > {alpha}, distribution is normal")

    qqplot(data, line='s', dist=norm)

shapiro_qqplot(filtered_women['total_goals_scored'], 0.05)
shapiro_qqplot(filtered_men['total_goals_scored'], 0.05)


In [ ]:
# Perform Wilcoxon-Mann-Whitney (Mann-Whitney U) at 10% significance level - non-normal counter part of two-sample independent t-tests
stat, p_val = mannwhitneyu(filtered_women['total_goals_scored'], filtered_men['total_goals_scored'], alternative="greater")
alpha = 0.01
if p_val > alpha:
    result = "fail to reject"
    print("Fail to reject null hypothesis (There is no significant difference between goal scores of men and women (p>0.1)).")
else:
    result = 'reject'
    print("Reject null hypothesis (There is a significant difference between goal scores of men and women (p<0.1)).")

result_dict = {"p_val": p_val, "result": result}
print(result_dict)